In [1]:
text = "WHAT DID YOU SAY"
text = text.lower()
print(text)
import random
from utils import *
#from ipynb.fs.full.zoecode import *
import re
from math import log
from math import isclose
import numpy

what did you say


## Some basic definitions

I defined alpha_list as a list of all the letters in the alphabet with a space included. This helps assign each letter an index and is used in many definitions.

In [8]:
print(alpha_list)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ']


The function `char_to_num`, takes a piece of text (a string of lowercase letters) and returns a list of the numerical values for each number, based on the indices in alpha_list. 

The function after that, `num_to_char`, reverses the effect, going from a series of numbers (ranged 0 - 26) to a string of lowercase letters returned as a list.

In [9]:
char_to_num("abc")

[0, 1, 2]

In [10]:
num_to_char([24,22,0])

['y', 'w', 'a']

# A caesar cipher

A caesar cipher is just a shift cipher.
The function `encode_caesar_cipher` takes a piece of text and a key, and shifts each letter forwards by the key. For example, if the key was 3, an 'a' would become a 'd', a 'b' would become a 'e' , and so on. The function `decode_caesar_cipher` works much the same way, but instead of shifting the letters forwards, it shifts them backwards, effectively reversing the encoding of the cipher. This only works if you use the same key to encode and decode, however.

In [11]:
key = 3
ciphertext = encode_caesar_cipher('hello', key)
plaintext = decode_caesar_cipher(ciphertext,key)
print(ciphertext)
print(plaintext)

['k', 'h', 'o', 'o', 'r']
['h', 'e', 'l', 'l', 'o']


## Aristocrat cipher

An aristocrat cipher is a random substitution cipher. Each letter in the alphabet encodes to another letter. In this code, I often abbreviate aristocrat to 'aris'.
`make_rand_aris_key` generates a random 'key' to be used in encoding and decoding an aristocrat. The key is a list of numbers. The index of the number is the letter that codes for another, and the number in the index is the number of the letter that it codes for. For example: index 0 represents 'a'. If the number at index 0 is 1, then whenever an 'a' comes up in a text, it would be encoded into a 'b', because b is the letter at index 1 of alpha_list.

The next two ciphers are similar to the encode and decode caesar ciphers. Instead of just shifting each letter, however, they use the key to encode and decode the letters.

In [2]:
key = make_rand_aris_key()
ciphertext = encode_aris('hello', key)
plaintext = decode_aris(ciphertext, key)
print(ciphertext)
print(plaintext)

['n', 'f', 'u', 'u', 'r']
['h', 'e', 'l', 'l', 'o']


In [80]:

bible_pair_count = count_letter_pairs(bible_letters, 0.5)
bible_matrix = compute_transition_matrix(bible_pair_count)

In [230]:
index1 = alpha_list.index("a")
index2 = alpha_list.index("e")
print(test_text)
text_letter_count = count_letters(test_text)
text_pair_count = count_letter_pairs(test_text)


one = find_pair_log_likelihood( test_text, bible_letter_percent, bible_matrix)
two = find_pair_log_likelihood(changed_text, bible_letter_percent, bible_matrix)
diff = (two - one)

start_key = list(alpha_list)
num_key = char_to_num(start_key)
index1 = alpha_list.index("a")
index2 = alpha_list.index("e")


faster_calc = (calculate_likelihood_change(test_text, text_pair_count, bible_letter_percent, bible_matrix,\
                                  index1, index2, num_key))
print(faster_calc)

assert isclose(diff, faster_calc), 'should be equal'
print("test passed")

['h', 'e', 'l', 'l', 'o', ' ', 'h', 'o', 'w', ' ', 'a', 'r', 'e', ' ', 'y', 'o', 'u']
hallo how era you
5.411230978190019
test passed


In [82]:
def find_letter_num(text, letter_count):
    return len(text) - letter_count[26]

In [87]:
def get_percent_lists(text_letters):
    letter_count = count_letters(text_letters)
    letter_num = find_letter_num(text_letters, letter_count)
    percent_list = normalize_counts_no_spaces(letter_count)
    pair_count = count_letter_pairs(text_letters)
    transition_matrix = compute_transition_matrix(pair_count)
    return percent_list, transition_matrix

In [89]:
german_letters = make_letters('german_text.txt')
german_list, german_pair_count = get_percent_lists(german_letters)
english_letters = make_letters('bible.txt')
english_list, english_pair_count = get_percent_lists(english_letters)
print(find_language('hallo wie gehts du', german_list, german_pair_count, english_list, english_pair_count))

The text is most likely German


In [90]:
def decode_aris_no_key(ciphertext, start_key, guess_num = 3000):
    count = 0
    num_key = start_key
    new_num_key = list(num_key)
    for i in range(guess_num):
        index1 = random.randint(0, 25)
        index2 = random.randint(0, 25)
        plaintext = decode_aris(ciphertext, num_key)
        new_num_key[index1], new_num_key[index2] = num_key[index2], num_key[index1]
        new_plaintext = decode_aris(ciphertext, new_num_key)
        plaintext_likely = find_pair_log_likelihood(plaintext, percent_list, transition_matrix)
        new_plaintext_likely = find_pair_log_likelihood(new_plaintext, percent_list, transition_matrix)
        if new_plaintext_likely > plaintext_likely:
            num_key = list(new_num_key)
            pair_count[index1], pair_count[index2] = pair_count[index2], pair_count[index1]
            count += 1
        else:
            new_num_key = list(num_key)
        plaintext = decode_aris(ciphertext, num_key)
        log_likelihood = find_pair_log_likelihood(plaintext, percent_list, transition_matrix)
    return (''.join(decode_aris(ciphertext, num_key))), count, log_likelihood

In [233]:
def switch_row_and_columns(text_pair_count, index1, index2):
    text_1 = list(text_pair_count[index1])
    text_pair_count[index1] = list(text_pair_count[index2])
    text_pair_count[index2] = list(text_1)
    for i in range(len(text_pair_count)):
        value_i1 = int(text_pair_count[i][index1])
        value_i2 = int(text_pair_count[i][index2])
        text_pair_count[i][index1] = value_i2
        text_pair_count[i][index2] = value_i1  

In [217]:
def decode_aris_faster(ciphertext, start_key, text_pair_count, ref_letter_percent, \
                       ref_transition_matrix, guess_num = 3000):
    count = 0
    num_key = list(start_key)
    for i in range(guess_num):
        index1 = random.randint(0, 25)
        index2 = random.randint(0, 25)
        while index1 == index2:
            index1 = random.randint(0, 25)
        plaintext = decode_aris(ciphertext, num_key)
        change = calculate_likelihood_change(plaintext, text_pair_count, ref_letter_percent,\
                                             ref_transition_matrix, index1, index2, num_key)
        print(change)
        print(num_key)
        if change > 0:
            num_1 = int(num_key[index1])
            num_key[index1] = int(num_key[index2])
            num_key[index2] = num_1
            count += 1
            print(count)
            print(num_key)
            switch_row_and_columns(text_pair_count, index1, index2)
        print(num_key)
    return(''.join(decode_aris(ciphertext, num_key)))

        

[[1, 3, 2], [7, 9, 8], [4, 6, 5]]
[[9, 7, 8], [3, 1, 2], [6, 4, 5]]


In [214]:
ciphertext = encode_aris(eman_letters, make_rand_aris_key())
text_pair_count = count_letter_pairs(ciphertext)
letter_count = count_letters(bible_letters)
ref_pair_count = count_letter_pairs(bible_letters, 0.5)
ref_percent_list = normalize_counts_no_spaces(letter_count)
ref_transition_matrix = compute_transition_matrix(ref_pair_count)
start_key = make_start_key(ciphertext, ref_percent_list)

In [218]:
print(''.join(decode_aris(ciphertext, start_key)))
plaintext = (decode_aris_faster(ciphertext, start_key, text_pair_count, ref_percent_list, \
                        ref_transition_matrix))
#print(find_pair_log_likelihood(plaintext, ref_percent_list, ref_transition_matrix))
#print(find_pair_log_likelihood(eman_letters, ref_percent_list, ref_transition_matrix))

jhamhsw  h tshanusiytioabw tre ysenideat of tre maited nthten of hgesiuhh ysoulhghtioapresehn oa tre tpeatwneuoad dhw of neytegbes ia tre wehs of oms losd oae tromnhad eicrt rmadsed had niktwtpo h ysoulhghtioa phn innmed bw tre ysenideat of tre maited nthten uoathiaiac hgoac otres triacn tre follopiac to pittrht oa tre fisnt dhw of jhamhsw ia tre wehs of oms losd oae tromnhad eicrt rmadsed had niktwtrsee hll yesnoan reld hn nlhven pitria haw nthte os denicahted yhst of h nthte tre yeoyle preseof nrhll trea be ia sebellioa hchiant tre maited nthten nrhll be trea treauefosphsd had foseves fsee had tre ekeumtive covesageat of tre maited nthten iaulmdiac tre gilithsw had ahvhl hmtrositw treseof pill seuocaixe had ghiathia tre fseedog of nmur yesnoan had pill do ao hut os hutn to seysenn nmur yesnoan os haw of treg ia haw effostn trew ghw ghze fos treis hutmhl fseedogtrht tre ekeumtive pill oa tre fisnt dhw of jhamhsw hfosenhid bw ysoulhghtioa denicahte tre nthten had yhstn of nthten if haw

In [219]:
plaintext

'qglwgmx  g rmglbkmupruvlfx rys pmsbuislr vt rys wlursi brgrsb vt ghsmukgg pmvkzghgruvlcysmsgb vl rys rcslrxbskvli igx vt bsprshfsm ul rys xsgm vt vwm zvmi vls ryvwbgli sunyr ywlimsi gli budrxrcv g pmvkzghgruvl cgb ubbwsi fx rys pmsbuislr vt rys wlursi brgrsb kvlrgululn ghvln vrysm ryulnb rys tvzzvculn rv currygr vl rys tumbr igx vt qglwgmx ul rys xsgm vt vwm zvmi vls ryvwbgli sunyr ywlimsi gli budrxrymss gzz psmbvlb yszi gb bzgjsb curyul glx brgrs vm isbunlgrsi pgmr vt g brgrs rys psvpzs cysmsvt bygzz rysl fs ul msfszzuvl gngulbr rys wlursi brgrsb bygzz fs rysl ryslkstvmcgmi gli tvmsjsm tmss gli rys sdskwrujs nvjsmlhslr vt rys wlursi brgrsb ulkzwiuln rys huzurgmx gli lgjgz gwryvmurx rysmsvt cuzz mskvnluos gli hgulrgul rys tmssivh vt bwky psmbvlb gli cuzz iv lv gkr vm gkrb rv mspmsbb bwky psmbvlb vm glx vt rysh ul glx sttvmrb rysx hgx hges tvm rysum gkrwgz tmssivhrygr rys sdskwrujs cuzz vl rys tumbr igx vt qglwgmx gtvmsbgui fx pmvkzghgruvl isbunlgrs rys brgrsb gli pgmrb vt brgrsb ut gl

In [220]:
print(find_pair_log_likelihood(plaintext, ref_percent_list, ref_transition_matrix))
print(find_pair_log_likelihood(decode_aris(eman_letters, start_key), ref_percent_list, ref_transition_matrix))

-20639.119499883032
-20279.34705856185


In [94]:
'''
right = 0
wrong = 0
eman_letters = make_letters('Emancipation_Proclamation.txt')
ciphertext = (encode_aris(eman_letters, make_rand_aris_key()))
bible_letters = make_letters('bible.txt')
bible_percent_list, bible_transition_matrix = get_percent_lists(bible_letters)
for i in range(100):
    decoded_text, count, log_likelihood = decode_aris_no_key(ciphertext, make_start_key(ciphertext, bible_percent_list))
    if (log_likelihood) == (find_pair_log_likelihood(eman_letters, percent_list, transition_matrix)):
        right += 1
    else:
        wrong += 1
print('right: ' + str(right))
'''

"\nright = 0\nwrong = 0\neman_letters = make_letters('Emancipation_Proclamation.txt')\nciphertext = (encode_aris(eman_letters, make_rand_aris_key()))\nbible_letters = make_letters('bible.txt')\nbible_percent_list, bible_transition_matrix = get_percent_lists(bible_letters)\nfor i in range(100):\n    decoded_text, count, log_likelihood = decode_aris_no_key(ciphertext, make_start_key(ciphertext, bible_percent_list))\n    if (log_likelihood) == (find_pair_log_likelihood(eman_letters, percent_list, transition_matrix)):\n        right += 1\n    else:\n        wrong += 1\nprint('right: ' + str(right))\n"

In [ ]:
print('right: ' + str(right))

In [ ]:
bible_letters = make_letters('bible.txt')
crime_letters = make_letters('crime_and_punishment.txt')

#For the first case, text = bible_letters for all
letter_count = count_letters(bible_letters)
letter_num = find_letter_num(bible_letters, letter_count)
percent_list = normalize_counts_no_spaces(letter_count)
pair_count = count_letter_pairs(bible_letters)
transition_matrix = compute_transition_matrix(pair_count)

In [ ]:
#print(transition_matrix)
print(transition_matrix[alpha_list.index(" ")][alpha_list.index(" ")])
print(sum(pair_count[alpha_list.index('s')]))
#0.0003554540226212538

In [ ]:
plaintext = 'queen'
plaintext = (plaintext.lower())
ciphertext = encode_caesar_cipher(plaintext, 3)
solutions = []
#print(find_solution_brute(ciphertext))
solutions = (find_solution_brute_pairs(ciphertext))

In [ ]:
caesar_ciphertext = "alex"
solution_list = ordered_solutions(caesar_ciphertext, find_solution_brute)
print("Solutions using single letter frequencies")
for i in range(26):
    print(solution_list[i])
solution_list = ordered_solutions(caesar_ciphertext)
print("Solutions using letter pair frequencies")
for i in range(26):
    print(solution_list[i])

In [168]:
def make_start_key(text, ref_percent_list):
    start_key = []
    text_letter_count = count_letters(text)
    text_percent_list = normalize_counts_no_spaces(text_letter_count)
    sorted_text_percent_list = list(text_percent_list)
    sorted_text_percent_list.sort(reverse = True)
    sorted_ref_percent_list = list(ref_percent_list)
    sorted_ref_percent_list.sort(reverse = True)
    for i in range(26):
        start_key.append(0)
    for i in range(26):
        key_value = ref_percent_list.index(sorted_ref_percent_list[i])
        key_index = text_percent_list.index(sorted_text_percent_list[i])
        start_key[key_index] = key_value
        text_percent_list[key_index] = -1
    return start_key

In [170]:
eman_letters = make_letters('Emancipation_Proclamation.txt')
#ciphertext = encode_aris(eman_letters, make_rand_aris_key())
ref_letter_count = count_letters(bible_letters, 0.5)
ref_letter_num = find_letter_num(bible_letters, ref_letter_count)
ref_percent_list = normalize_counts_no_spaces(ref_letter_count)
start_key = make_start_key(eman_letters, ref_percent_list)
right = 0
wrong = 0
plaintext = (''.join(decode_aris(ciphertext, start_key)))
print(start_key)
#print(plaintext)

[13, 1, 6, 3, 4, 5, 12, 0, 8, 23, 9, 11, 20, 18, 14, 22, 16, 7, 17, 19, 2, 21, 24, 25, 15, 10]
sfalfwn  f jwfazmwrtjrqaon jub twbzrebaj qg jub larjbe zjfjbz qg fdbwrmff twqmhfdfjrqayubwbfz qa jub jybajnzbmqae efn qg zbtjbdobw ra jub nbfw qg qlw hqwe qab juqlzfae brpuj ulaewbe fae zrijnjyq f twqmhfdfjrqa yfz rzzlbe on jub twbzrebaj qg jub larjbe zjfjbz mqajfrarap fdqap qjubw jurapz jub gqhhqyrap jq yrjjufj qa jub grwzj efn qg sfalfwn ra jub nbfw qg qlw hqwe qab juqlzfae brpuj ulaewbe fae zrijnjuwbb fhh tbwzqaz ubhe fz zhfxbz yrjura fan zjfjb qw ebzrpafjbe tfwj qg f zjfjb jub tbqthb yubwbqg zufhh juba ob ra wbobhhrqa fpfrazj jub larjbe zjfjbz zufhh ob juba jubambgqwyfwe fae gqwbxbw gwbb fae jub bibmljrxb pqxbwadbaj qg jub larjbe zjfjbz ramhlerap jub drhrjfwn fae afxfh fljuqwrjn jubwbqg yrhh wbmqparkb fae dfrajfra jub gwbbeqd qg zlmu tbwzqaz fae yrhh eq aq fmj qw fmjz jq wbtwbzz zlmu tbwzqaz qw fan qg jubd ra fan bggqwjz jubn dfn dfvb gqw jubrw fmjlfh gwbbeqdjufj jub bibmljrxb yrhh qa jub

In [129]:
len(ref_percent_list)

26

In [ ]:
print(start_key)
print(start_key.index(16))